In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='2'

In [2]:
import numpy as np
import cv2 as cv
import tensorflow as tf
from tensorflow_examples.models.pix2pix import pix2pix

In [3]:
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [4]:
MEDIA_PATH = os.path.join('/media', 'disk2', 'amaltsev', 'car_lp_generator')

BUFFER_SIZE = 1000

In [5]:
rendered_new_path = os.path.join(MEDIA_PATH, 'style_dst_sq')

dataset = tf.data.Dataset.from_tensor_slices([os.path.join(rendered_new_path, img) for img in sorted(os.listdir(rendered_new_path))[1:]])

In [6]:
def normalize(image):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    return image

In [7]:
def load_preprocess_image_test(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    
#     image = tf.image.resize(image, [256, 256], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    image = normalize(image)
    return image

In [8]:
dataset = dataset.map(load_preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().batch(1)

In [9]:
OUTPUT_CHANNELS = 3

generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

In [10]:
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [11]:
checkpoint_path = os.path.join(MEDIA_PATH, 'checkpoints', 'train')

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


In [12]:
import time

In [13]:
dataset_iterator = iter(dataset)

i = 0
start_time = time.time()

for img_name in sorted(os.listdir(rendered_new_path))[1:]:
    
#     print('processing', img_name)
    
    image = next(dataset_iterator)
    
#     if os.path.exists(os.path.join(MEDIA_PATH, 'final', img_name)):
#         continue
    
#     new_mask = cv.imread(os.path.join(MEDIA_PATH, 'new_masks', img_name))
#     h1_bound, h2_bound, w1_bound, w2_bound = -1, -1, new_mask.shape[1], -1
#     print(new_mask.shape)
    
#     for i in range(new_mask.shape[0]):
#         for j in range(new_mask.shape[1]):
#             if (new_mask[i][j] > np.array([200, 200, 200])).all():
#                 if h1_bound == -1:
#                     h1_bound = i
                
#                 h2_bound = i
                
#                 if j < w1_bound:
#                     w1_bound = j
                
#                 if j > w2_bound:
#                     w2_bound = j
    
#     print(h1_bound, h2_bound, w1_bound, w2_bound)
    
#     cropped = image[:, h1_bound : h2_bound, w1_bound : w2_bound, :]
    cropped = tf.image.resize(image, [256, 256], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
    transferred = generator_f(cropped)[0]
    transferred = (transferred + 1) * 127.5
    transferred = tf.cast(transferred, tf.uint8)
    tf.io.write_file(os.path.join(MEDIA_PATH, 'transferred_cyclegan', img_name), tf.io.encode_jpeg(transferred))
    
    if i % 100 == 0:
        print(i)
        print(time.time() - start_time)
    i += 1
    
#     transferred = tf.image.resize(transferred, [h2_bound - h1_bound + 1, w2_bound - w1_bound + 1], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    
#     removed_name = img_name.split('.')[0][:-10] + '.jpg'
#     final = cv.imread(os.path.join(MEDIA_PATH, 'removed_plates', removed_name))
    
#     for i in range(h1_bound, h2_bound + 1):
#         for j in range(w1_bound, w2_bound + 1):
#             if (new_mask[i][j] > np.array([200, 200, 200])).all():
#                 final[i][j] = transferred[i - h1_bound][j - w1_bound]
    
#     cv.imwrite(os.path.join(MEDIA_PATH, 'final', img_name), final)

0
0.22086596488952637
100
25.21204948425293
200
46.80657434463501
300
75.59317803382874
400
98.62221622467041
500
119.50025939941406
600
143.60392022132874
700
163.05777215957642
800
188.90511274337769
900
212.26556944847107
1000
233.81289076805115


KeyboardInterrupt: 